In [1]:
from ultralytics import YOLO
import cv2

In [2]:
model = YOLO('runs/segment/train10/weights/best.pt') # load model

In [16]:
# Code Overview
# Opens video -> runs segmentation model on each frame -> draws predictions onto frame - > saves output video of segmented frames

In [22]:
# Open video
cap = cv2.VideoCapture("video1.mp4", cv2.CAP_FFMPEG) # replace file with desired video to segment
ret, frame = cap.read()
if not ret or frame is None:
    cap.release()
    raise ValueError("Failed to read video")

# Run one prediction to get output size
results = model.predict(source=frame, verbose=False)
annotated = results[0].plot()
height, width = frame.shape[:2]

# Set up video writer (original size)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter("video-seg.mp4", fourcc, fps, (width, height)) # output video save file path

# Rewind video to beginning
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# Create resizable window
cv2.namedWindow("Segmented Frame", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Segmented Frame", 300, 400)  # Resize for screen

frame_count = 0

# Process and display
while cap.isOpened():
    ret, frame = cap.read()
    if not ret or frame is None:
        break

    results = model.predict(source=frame, verbose=False) # runs segmentation on current frame
    annotated = results[0].plot() # draws predictions onto frame

    out.write(annotated)  # Save full-size frame
    frame_count += 1
    resized = cv2.resize(annotated, (300, 400))  # Resize for display
    cv2.imshow("Segmented Frame", resized)

    key = cv2.waitKey(1)
    if key == ord('q'): # pressing q closes the preview window
        cv2.destroyWindow("Segmented Frame")

print("Frames written:", frame_count)

cap.release()
out.release()
cv2.destroyAllWindows()

Frames written: 628
